In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"apepexz","key":"789278ce01e65dfc203e37ba88bb71a5"}'}

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database

Dataset URL: https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database
License(s): CC0-1.0
anime-recommendations-database.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip anime-recommendations-database.zip

Archive:  anime-recommendations-database.zip
replace anime.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
anime = pd.read_csv('anime.csv')
rating = pd.read_csv('rating.csv')
fraction_to_keep = 1 / 150
random_mask = np.random.rand(len(rating)) < fraction_to_keep
rating_filtered = rating[random_mask]
rating_filtered.to_csv('rating_filtered.csv', index=False)
print(rating_filtered.head())
print(anime.head())

     user_id  anime_id  rating
4          1       241      -1
536        5     10172       1
661        5     18497       5
684        5     20159       7
694        5     20787       3
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  

In [ ]:
print(anime.shape)
print(rating_filtered.shape)

(12294, 7)
(52030, 3)


In [ ]:
Yawal = rating_filtered.pivot_table(
    index='anime_id',
    columns='user_id',
    values='rating',
    fill_value=-1
)

In [ ]:
R = Yawal.applymap(lambda x: 0 if x == -1 else 1)
Y = Yawal.applymap(lambda x: 0 if x == -1 else x)
print(R.head())
print(Y.head())

<ipython-input-13-428bceb93221>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  R = Yawal.applymap(lambda x: 0 if x == -1 else 1)
<ipython-input-13-428bceb93221>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Y = Yawal.applymap(lambda x: 0 if x == -1 else x)


user_id   1      5      7      13     14     17     18     23     24     \
anime_id                                                                  
1             0      0      0      0      0      0      0      0      0   
5             0      0      0      0      0      0      0      0      0   
6             0      0      0      0      0      0      0      0      0   
7             0      0      0      0      0      0      0      0      0   
8             0      0      0      0      0      0      0      0      0   

user_id   28     ...  73485  73495  73499  73500  73502  73503  73507  73510  \
anime_id         ...                                                           
1             0  ...      0      0      0      0      0      0      0      0   
5             0  ...      0      0      0      0      0      0      0      0   
6             0  ...      0      0      0      0      0      0      0      0   
7             0  ...      0      0      0      0      0      0      0     

'\n# Menghitung R: -1 menjadi 0, lainnya menjadi 1\nR = np.where(Yawal == -1, 0, 1)\n\n# Menghitung Y: -1 menjadi 0, lainnya tetap\nY = Yawal.replace(-1, 0)\n'

In [ ]:
from google.colab import files

R.to_csv('R_result.csv', index=False)
Y.to_csv('Y_result.csv', index=False)


files.download('R_result.csv')
files.download('Y_result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This cost function was obtained from https://www.coursera.org/learn/unsupervised-learning-recommenders-reinforcement-learning

Just like any other machine learning model's lost functions, this lost function is used to obtain the optimal amount of W and b

In [ ]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    nm, nu = Y.shape
    J = 0
    ### START CODE HERE ###
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += r * np.square((np.dot(w,x) + b_j - y ))
    J += (lambda_) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    J = J/2
    ### END CODE HERE ###

    return J

In [ ]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [ ]:
num_movies, num_users = Y.shape
num_features = 100
Ynorm, Ymean = normalizeRatings(Y, R)

tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [ ]:
def normalizeRatings(Y, R):
    """
    Preprocess data by subtracting mean rating for every movie (every row).
    Only include real ratings R(i,j)=1.
    [Ynorm, Ymean] = normalizeRatings(Y, R) normalized Y so that each movie
    has a rating of 0 on average. Unrated moves then have a mean rating (0)
    Returns the mean rating in Ymean.
    """
    # Convert the Series to NumPy array before reshape
    Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).to_numpy().reshape(-1,1)
    Ynorm = Y - np.multiply(Ymean, R)
    return(Ynorm, Ymean)

In [ ]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 6756.9
Training loss at iteration 20: 6062.0
Training loss at iteration 40: 5579.8
Training loss at iteration 60: 5242.5
Training loss at iteration 80: 5005.2
Training loss at iteration 100: 4837.2
Training loss at iteration 120: 4717.1
Training loss at iteration 140: 4629.9
Training loss at iteration 160: 4565.5
Training loss at iteration 180: 4516.9


As expected, the result is indeed a bit off since the data was reduced by 1/150 because the RAM and Disk maximum capacity in google collab ran out if i used all of the data.

In [ ]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

pm = p + Ymean

my_predictions = pm[:,0]

ix = tf.argsort(my_predictions, direction='DESCENDING')


for i in range(100):
    j = ix[i].numpy()
    anime_id = Yawal.index[j]
    anime_row = anime[anime['anime_id'] == anime_id].iloc[0]

    original_rating = anime_row['rating'] if 'rating' in anime_row else 'N/A'

    print(f"Predicting rating {my_predictions[j]:0.2f} for anime {anime_row['name']} (Original rating: {original_rating})")


Predicting rating 9.92 for anime Hourou Musuko Specials (Original rating: 7.7)
Predicting rating 9.92 for anime Nee Summer! (Original rating: 7.08)
Predicting rating 9.92 for anime Tamayura: Sotsugyou Shashin Part 3 - Akogare (Original rating: 7.88)
Predicting rating 9.92 for anime Sorette♡Dakara ne! (Original rating: 5.15)
Predicting rating 9.92 for anime Namiuchigiwa no Muromi-san OVA (Original rating: 6.83)
Predicting rating 9.92 for anime Wonderful Rush (Original rating: 7.4)
Predicting rating 9.92 for anime Nee, Chanto Shiyou Yo! (Original rating: 7.09)
Predicting rating 9.92 for anime Tentai Senshi Sunred (Original rating: 7.96)
Predicting rating 9.92 for anime Macross Dynamite 7 (Original rating: 6.97)
Predicting rating 9.92 for anime Shakkin Shimai (Original rating: 6.15)
Predicting rating 9.92 for anime Shijou Saikyou no Deshi Kenichi Specials (Original rating: 7.35)
Predicting rating 9.92 for anime Detective Conan Movie 19: The Hellfire Sunflowers (Original rating: 7.77)
Pred